# Generators i Python
**Eksamenspræsentation - Introduktion til Python**

---

## Hvad er generators?

Generators er en effektiv måde at skabe iteratorer i Python. De gør det muligt at:
- **Spare hukommelse** - genererer værdier on-the-fly
- **Spare tid** - kun beregner værdier når de er nødvendige (lazy evaluation)
- **Skrive mere læselig kode** - simplere end iterator-klasser

## 1. Iterator-klasser (Den traditionelle måde)

Først ser vi på hvordan vi kan lave en iterator ved at implementere `__iter__()` og `__next__()` metoderne:

In [ ]:
from time import sleep

class Compute:
    """Iterator-klasse der genererer tal fra 0 til 9"""
    
    def __iter__(self):
        # __iter__ initialiserer iteratoren og returnerer sig selv
        self.last = 0
        return self
    
    def __next__(self):
        # __next__ returnerer næste værdi i sekvensen
        rv = self.last
        self.last += 1
        
        # Når vi når 10, stopper vi iterationen
        if self.last > 10:
            raise StopIteration()
        
        sleep(0.5)  # Simulerer tung beregning
        return rv

# Demonstration
print("Iterator-klasse eksempel:")
for i in Compute():
    print(i, end=" ")

### Problem med Iterator-klasser:
- Kræver meget boilerplate code
- Mindre læselig
- Håndtering af tilstand kan være kompleks

## 2. Generator-funktioner (Den moderne måde)

Generator-funktioner bruger `yield` i stedet for `return`. Hver gang `yield` kaldes, "pauser" funktionen og gemmer sin tilstand:

In [ ]:
def compute():
    """Generator-funktion der gør det samme som Compute-klassen"""
    # yield gør funktionen til en generator
    # Funktionen "pauser" her og husker sin tilstand
    for i in range(10):
        yield i

# Demonstration
print("Generator-funktion eksempel:")
for i in compute():
    print(i, end=" ")

print("\n\nGenerator objektet:", compute())

### Fordele ved generator-funktioner:
✅ Meget kortere og mere læselig kode

✅ Python håndterer tilstand automatisk

✅ Automatisk StopIteration når funktionen slutter

## 3. Generator Expressions (Den korteste måde)

Generator expressions ligner list comprehensions, men bruger parenteser () i stedet for kantede parenteser []:

In [ ]:
# List comprehension - opretter hele listen i hukommelsen
list_comp = [i for i in range(10)]
print("List comprehension:", list_comp)
print("Type:", type(list_comp))

# Generator expression - genererer værdier on-the-fly
gen_exp = (i for i in range(10))
print("\nGenerator expression:", gen_exp)
print("Type:", type(gen_exp))

# Konverter til liste for at se værdierne
print("Værdier:", list(gen_exp))

## 4. Praktisk eksempel: Hukommelseseffektivitet

Lad os demonstrere hvorfor generators er vigtige med et realistisk eksempel:

In [ ]:
import sys
import random

names = ['John', 'Corey', 'Adam', 'Steve', 'Rick', 'Thomas']
majors = ['Math', 'Engineering', 'CompSci', 'Arts', 'Business']

def students_list(num_students):
    """Returnerer en liste - gemmer ALLE studerende i hukommelsen"""
    result = []
    for student_id in range(num_students):
        student = {
            'id': student_id,
            'name': random.choice(names),
            'major': random.choice(majors)
        }
        result.append(student)
    return result

def students_generator(num_students):
    """Returnerer en generator - gemmer KUN én studerende ad gangen"""
    for student_id in range(num_students):
        student = {
            'id': student_id,
            'name': random.choice(names),
            'major': random.choice(majors)
        }
        yield student  # 'yield' gør det til en generator

# Sammenligning af hukommelsesforbrug
list_students = students_list(10000)
gen_students = students_generator(10000)

print(f"Liste størrelse: {sys.getsizeof(list_students):,} bytes")
print(f"Generator størrelse: {sys.getsizeof(gen_students):,} bytes")
print(f"\nHukommelsesbesparelse: {sys.getsizeof(list_students) / sys.getsizeof(gen_students):.1f}x")

In [ ]:
# Vi kan stadig iterere over generatoren som en liste
gen_students = students_generator(5)
print("Første 5 studerende fra generator:")
for student in gen_students:
    print(f"  {student}")

## 5. Avanceret eksempel: Custom Range Iterator

Pythons `range()` er faktisk ikke en funktion - det er en klasse! Lad os genskabe den:

In [ ]:
class MyRange:
    """Iterator-klasse der efterligner range()"""
    
    def __init__(self, start, stop, step=1):
        self.start = start
        self.stop = stop
        self.step = step
    
    def __iter__(self):
        # Initialiser den aktuelle værdi
        self.current = self.start
        return self
    
    def __next__(self):
        # Tjek om vi skal stoppe
        if self.current >= self.stop:
            raise StopIteration
        
        # Gem nuværende værdi, increment, og returner
        result = self.current
        self.current += self.step
        return result

# Test vores custom range
print("MyRange iterator:")
for i in MyRange(1, 10, 2):
    print(i, end=" ")

In [ ]:
def my_range_generator(start, stop, step=1):
    """Generator-funktion version - meget simplere!"""
    current = start
    while current < stop:
        yield current
        current += step

# Test generator versionen
print("\nmy_range_generator:")
for i in my_range_generator(1, 10, 2):
    print(i, end=" ")

## 6. Iterabel Student Collection

Et praktisk eksempel hvor vi gør en klasse iterabel:

In [ ]:
class Student:
    """Simpel Student klasse"""
    def __init__(self, name, cpr):
        self._name = name
        self._cpr = cpr
    
    def __str__(self):
        return f'{self._name}, {self._cpr}'

class PythonStudents:
    """Container klasse for studerende - iterabel!"""
    
    def __init__(self):
        self.students = []
    
    def add_student(self, student):
        """Tilføj en studerende til samlingen"""
        self.students.append(student)
    
    def __iter__(self):
        """Gør klassen iterabel - returner en generator!"""
        # Vi kan bruge en generator expression her
        return (student._name for student in self.students)

# Demonstration
python_class = PythonStudents()
python_class.add_student(Student("Anna", "123456-7890"))
python_class.add_student(Student("Peter", "234567-8901"))
python_class.add_student(Student("Marie", "345678-9012"))

print("Studerende i Python klassen:")
for name in python_class:
    print(f"  - {name}")

## Konklusion

### De tre måder at skabe iteratorer:

1. **Iterator-klasser** (`__iter__` & `__next__`)
   - Mest verbose, men mest kontrollerbar
   - Bruges når du har brug for kompleks tilstandsstyring

2. **Generator-funktioner** (`yield`)
   - Balance mellem læsbarhed og funktionalitet
   - Perfekt til de fleste use cases

3. **Generator expressions** (`(x for x in ...)`)
   - Kortest og mest concise
   - Ideel til simple transformationer

### Hvorfor er generators vigtige?
- 🚀 **Performance**: Lazy evaluation sparer CPU-tid
- 💾 **Hukommelse**: Kun én værdi i hukommelsen ad gangen
- 📖 **Læsbarhed**: Klarere intentioner end iterator-klasser
- ♾️ **Uendelige sekvenser**: Kan repræsentere uendelige datastrømme